In [1]:
import pandas as pd
import os
import numpy as np 
import random

import random
import pandas as pd
import numpy as np
import pickle
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler

# Read Data 

In [16]:
raw_path = './raw/'
olist_rcm = pd.read_csv(raw_path + 'Olist_Recommendation_Dataset.csv')


# Data overview

In [17]:
olist_rcm.head()

,Customer State,Review Score,Purchase Timestamp,Purchase Date,Monetary,Customer Segment,Marketing Action,Customer_ID,Order_ID,Product_ID,product_name,aisle,department,Product Category
0,RJ,5,9/13/2017 8:59,9/13/2017 0:00,281.30,Potential Loyalists,Cross Sell Recommendations and Discount coupons,15438,0,8446,Utility Lighter,more household,household,more household
1,GO,5,6/28/2017 11:52,6/28/2017 0:00,73.86,Lost Customers,Don't spend too much trying to re-acquire,22529,7229,8446,Utility Lighter,more household,household,more household
2,MG,4,5/18/2018 10:25,5/18/2018 0:00,108.32,Potential Loyalists,Cross Sell Recommendations and Discount coupons,17139,31508,8446,Utility Lighter,more household,household,more household
3,PR,5,8/1/2017 18:38,8/1/2017 0:00,472.06,Loyal Customers,Loyality programs;Cross Sell,33008,42252,8446,Utility Lighter,more household,household,more household
4,MG,5,8/10/2017 21:48,8/10/2017 0:00,1415.74,VVIP - Can't Loose Them,No Price Incentives; Offer Limited edition and...,41377,47321,8446,Utility Lighter,more household,household,more household


In [18]:
features = ['Customer_ID','product_name','Product_ID','Review Score']
olist_rcm = olist_rcm[features]
olist_rcm = olist_rcm.rename(columns={"Review Score":"ratings","Customer_ID":"customer_id","Product_ID":"product_id"})

In [19]:
from scipy.sparse import csr_matrix

In [20]:
olist_rcm = olist_rcm[:20000]

In [22]:
olist_rcm.customer_id.unique()

array([15438, 22529, 17139, ..., 24740, 11139, 37746], dtype=int64)

In [25]:
# pivot ratings into movie features
df_product_features = olist_rcm.reset_index().pivot_table(
    index='customer_id',
    columns='product_id',
    values='ratings'
).fillna(0)
# convert dataframe of movie features to scipy sparse matrix
mat_product_features = csr_matrix(df_product_features.values)

In [26]:
df_product_features.head()

product_id,63,69,151,161,185,219,226,238,255,311,...,32021,32050,32052,32083,32115,32134,32164,32170,32171,32203
customer_id,,,,,,,,,,,,,,,,,,,,,
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Buil Recommendations system

In [27]:
df_product_features.shape

(15530, 1170)

In [28]:
X = df_product_features.T
X.head()

customer_id,5,9,12,15,16,19,20,24,27,29,...,41410,41412,41413,41416,41418,41419,41421,41422,41424,41425
product_id,,,,,,,,,,,,,,,,,,,,,
63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
X.index[:-20]

Index([   63,    69,   151,   161,   185,   219,   226,   238,   255,   311,
       ...
       31567, 31575, 31585, 31605, 31607, 31631, 31654, 31668, 31671, 31692],
      dtype='int64', name='product_id', length=1150)

In [30]:
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(1170, 10)

In [31]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(1170, 1170)

In [32]:
correlation_matrix

array([[ 1.        ,  0.31545991,  0.8226531 , ...,  0.1692515 ,
         0.01447072,  0.14534405],
       [ 0.31545991,  1.        ,  0.41707688, ...,  0.22792024,
         0.3545751 ,  0.29873521],
       [ 0.8226531 ,  0.41707688,  1.        , ...,  0.19643681,
        -0.14536194,  0.34114784],
       ...,
       [ 0.1692515 ,  0.22792024,  0.19643681, ...,  1.        ,
         0.00147794,  0.10487798],
       [ 0.01447072,  0.3545751 , -0.14536194, ...,  0.00147794,
         1.        ,  0.1130078 ],
       [ 0.14534405,  0.29873521,  0.34114784, ...,  0.10487798,
         0.1130078 ,  1.        ]])

In [35]:
fave_prod = olist_rcm.groupby(['customer_id']).max()['product_id'].to_frame()

In [36]:
fave_prod = fave_prod.reset_index()
fave_prod

,customer_id,product_id
0,5,10630
1,9,6927
2,12,15675
3,15,27919
4,16,10630
...,...,...
15525,41419,13331
15526,41421,18573
15527,41422,15250
15528,41424,21641


In [42]:
def get_prod_id(customer_id):
    
    product_id = fave_prod[fave_prod.customer_id == customer_id]['product_id']
    return product_id
    
    
prd_id = get_prod_id(41422)
product_id = prd_id.iloc[0]
product_id

15250

In [43]:
product_names = list(X.index)
product_ID_Idx = product_names.index(product_id)
product_ID_Idx

529

In [44]:

correlation_product_ID = correlation_matrix[product_ID_Idx]
correlation_product_ID.shape

(1170,)

In [46]:
Recommend = list(X.index[correlation_product_ID > 0.70])
# Removes the item already bought by the customer
Recommend.remove(product_id) 
Recommend[0:20]

[69,
 602,
 1477,
 3417,
 4835,
 4875,
 4969,
 5821,
 9355,
 10766,
 11228,
 12089,
 12264,
 12712,
 16684,
 16929,
 16961,
 17518,
 18305,
 18421]

In [47]:
## Getting Product names froom prediction 

predictions = pd.DataFrame(Recommend[:20])
predictions.columns = ['product_id']
predictions

,product_id
0,69
1,602
2,1477
3,3417
4,4835
5,4875
6,4969
7,5821
8,9355
9,10766


In [49]:
predictions['Product Name'] = predictions.product_id.apply(lambda x : olist_rcm[olist_rcm.product_id == x]['product_name'].unique()[0])


In [54]:
predictions.head(10)

,product_id,Product Name
0,69,Vanilla with Almond Milk Iced Coffee
1,602,Banana Chocolate Chip
2,1477,Wild Mushroom Cauliflower Hempseed Burgers
3,3417,Chocolate Creme Cookies
4,4835,Strong Thai Sweet Chili Energy Bar
5,4875,Buffalo Wing Sauce
6,4969,Squirt Rub R 12 Floz 12 Pk 144 Floz
7,5821,Asian Noodles Teriyaki
8,9355,Creamy Italian Dressing
9,10766,Wholesome Gluten Free Flour


In [55]:
olist_rcm.to_csv(raw_path + 'Olist_db_ALS.csv', index=False)


In [56]:
X.to_csv('X_ALS.csv')

In [60]:
correlation_matrix.shape

(1170, 1170)

In [61]:
import json

with open('correlation_matrix_ALS.txt', 'w') as filehandle:
    json.dump(correlation_matrix.tolist(), filehandle)

In [62]:
olist_rcm = pd.read_csv(raw_path +'Olist_db_ALS.csv')


In [63]:
X = pd.read_csv('X_ALS.csv', index_col=0)
X

,5,9,12,15,16,19,20,24,27,29,...,41410,41412,41413,41416,41418,41419,41421,41422,41424,41425
product_id,,,,,,,,,,,,,,,,,,,,,
63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
X.columns[4000:4020]

Index(['10675', '10677', '10679', '10680', '10681', '10682', '10685', '10686',
       '10689', '10694', '10696', '10698', '10699', '10700', '10703', '10712',
       '10713', '10714', '10716', '10719'],
      dtype='object')

In [65]:
import json
with open('correlation_matrix_ALS.txt') as f:
    correlation_matrix = json.load(f)

In [67]:
correlation_matrix = np.array(correlation_matrix)

In [68]:
X.index

Index([   63,    69,   151,   161,   185,   219,   226,   238,   255,   311,
       ...
       32021, 32050, 32052, 32083, 32115, 32134, 32164, 32170, 32171, 32203],
      dtype='int64', name='product_id', length=1170)

In [85]:
def Recommendations_ALS(customer_id):
    #Olist_db_ = Olist_db_[Olist_db_.Customer_ID < 3600]
    fave_prod = olist_rcm.groupby(['customer_id']).max()['product_id'].to_frame()
    fave_prod = fave_prod.reset_index()

    #prd_id = get_prod_id(Customer_id)
    prd_id = fave_prod[fave_prod.customer_id == customer_id]['product_id']
    product_id = prd_id.iloc[0]
    product_names = list(X.index)
    product_ID_Idx = product_names.index(product_id)

    correlation_product_ID = correlation_matrix[product_ID_Idx]
    
    Recommend = list(X.index[correlation_product_ID > 0.70])
    # Removes the item already bought by the customer
    Recommend.remove(product_id) 
    
    ## Getting Product names froom prediction 

    predictions = pd.DataFrame(Recommend[:20])
    predictions.columns = ['product_id']

    predictions['Product Name'] = predictions.product_id.apply(lambda x : olist_rcm[olist_rcm.product_id == x]['product_name'].unique()[0])
    Recommendations = predictions[:10]
    return Recommendations

In [88]:
Recommendations = Recommendations_ALS(10000)
Recommendations[:10]

,product_id,Product Name
0,1969,Plain Cream Cheese
1,3366,"Soup Mix, Sausage & Lentil, Indiana Harvest"
2,3756,Blueberry Sparkling Water
3,3959,Signature Twin Size Mattress
4,4599,California Pinot Noir Red Wine
5,5828,Powder Fresh Roll-On Antiperspirant Deodorant
6,6305,Toasted Sesame Crackers
7,7927,No Touch Hand Soap Unit
8,8381,Roja Parfumes Luxe 3.4 oz
9,8774,Whole Grain Pita Bread
